In [1]:
import numpy as np
import pandas as pd
import time
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = "E:/CS_Master_Degree_UIUC/CS410_Text_Information_system/Project/Project Submission/CourseProject/Dataset/"
# Load transaction dataset
D = pd.read_csv(path + "DBLP2000.csv").astype('str')
D

,author,title
0,Lothar Breuer,Operator-Geometric Solutions for the M/G/k Que...
1,"Christopher Lusena, Judy Goldsmith, Martin Mun...",Nonapproximability Results for Partially Obser...
2,"János Komlós, Ali Shokoufandeh, Miklós Simonov...",The Regularity Lemma and Its Applications in G...
3,Vijay V. Vazirani,Primal-Dual Schema Based Approximation Algorit...
4,"Isabel Fernández-Anta, Eva Millán, José-Luis P...",Adaptation and Generation in a Web-Based Lisp ...
...,...,...
3999,"Marek Teichmann, Bud Mishra",Probabilistic Algorithms for Efficient Graspin...
4000,Robert E. Filman,Arachnoid Tourist: 2020 Hindsight.
4001,"Lassaâd Gannoun, Julien Francioli, Stanislav C...",Domain Name eXchange: A Mobile-Agent-Based Sha...
4002,"Heecheol Jeon, Charles J. Petrie, Mark R. Cutk...",JATLite: A Java Agent Infrastructure with Mess...


In [3]:
# Load context units
context_units = pd.read_csv(path + "DBLP2000_context_units.csv")
context_units

,pattern,transaction_index,pattern_type
0,Ralf Steinmetz,"[70, 1323, 1325, 3362]",author
1,Cheng-Wen Wu,"[194, 196, 288, 1909]",author
2,Thomas S. Huang,"[652, 835, 1012, 1123]",author
3,Maja J. Mataric,"[660, 669, 2266, 2271]",author
4,K. Suzanne Barber,"[707, 1311, 2129, 2444, 2448]",author
...,...,...,...
175,robocup,"[1381, 2200, 2202, 2819]",title
176,contextu,"[138, 156, 793, 993]",title
177,intension,"[272, 312, 990, 1360]",title
178,nest,"[636, 2289, 3280, 3402]",title


### Build weight matrix of transactions in the context unit space as bit vectors

In [4]:
D_size = len(D) # number of rows
C_size = len(context_units) # number of columns
# Initialize the weight matrix 
#transaction_w = np.zeros([C_size, D_size])
transaction_w = -1 * np.ones([C_size, D_size])
context_index = context_units['transaction_index'].apply(lambda row: row[1:-1].split(', ')).apply(lambda row: list(map(int, row)))
for i in range(C_size):
    context_ind = context_index.iloc[i]
    transaction_w[i, context_ind] = 1

In [5]:
transaction_weights = pd.DataFrame(transaction_w, index = context_units["pattern"])
transaction_weights

,0,1,2,3,4,5,6,7,8,9,...,3994,3995,3996,3997,3998,3999,4000,4001,4002,4003
pattern,,,,,,,,,,,,,,,,,,,,,
Ralf Steinmetz,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Cheng-Wen Wu,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Thomas S. Huang,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Maja J. Mataric,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
K. Suzanne Barber,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
robocup,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
contextu,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
intension,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
transaction_weights.to_csv(path + "transaction_weights.csv", index = False)

### Calculate similarity between the given transaction and the author 

In [6]:
# Get author list 
authors = context_units.iloc[0:14]["pattern"]
authors

0          Ralf Steinmetz
1            Cheng-Wen Wu
2         Thomas S. Huang
3         Maja J. Mataric
4       K. Suzanne Barber
5           Josef Kittler
6           Gerald Sommer
7        Edwin R. Hancock
8      Masaru Kitsuregawa
9        Roberto Gorrieri
10          Nancy E. Reed
11     Bharat K. Bhargava
12    Sanjay Kumar Madria
13           Bill Hancock
Name: pattern, dtype: object

In [7]:
# Get author weights
context_weights = pd.read_csv(path+ "Context_units_weights.csv")
author_weights = context_weights.iloc[:, 1:15]
author_weights

,Ralf Steinmetz,Cheng-Wen Wu,Thomas S. Huang,Maja J. Mataric,K. Suzanne Barber,Josef Kittler,Gerald Sommer,Edwin R. Hancock,Masaru Kitsuregawa,Roberto Gorrieri,Nancy E. Reed,Bharat K. Bhargava,Sanjay Kumar Madria,Bill Hancock
0,0.005007,0.003278,0.004386,0.003792,0.003458,0.003737,0.004386,0.004574,0.004262,0.004524,0.003418,0.003684,0.004867,0.002880
1,0.003278,0.005162,0.003851,0.003851,0.004071,0.002692,0.004453,0.004651,0.004330,0.004026,0.003477,0.004330,0.004330,0.003397
2,0.004386,0.003851,0.005007,0.003227,0.003458,0.003737,0.004386,0.004574,0.004262,0.004524,0.003418,0.004262,0.004867,0.003321
3,0.003792,0.003851,0.003227,0.004386,0.003458,0.002649,0.004386,0.004003,0.003684,0.003958,0.002903,0.003684,0.004262,0.002461
4,0.003458,0.004071,0.003458,0.003458,0.004684,0.003402,0.004003,0.004684,0.004434,0.004092,0.004092,0.003880,0.004434,0.002897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.003134,0.003743,0.003684,0.003134,0.003880,0.003086,0.003684,0.003880,0.003576,0.003835,0.003310,0.004138,0.004138,0.002759
176,0.003914,0.003382,0.003914,0.003331,0.003579,0.003858,0.004526,0.004734,0.003806,0.004684,0.004098,0.003806,0.005027,0.003018
177,0.004453,0.003910,0.003851,0.004453,0.004071,0.003230,0.004453,0.004651,0.004330,0.004601,0.003477,0.004330,0.004944,0.002947
178,0.003179,0.004390,0.003737,0.003179,0.003940,0.002609,0.003737,0.004502,0.004198,0.003895,0.003363,0.004198,0.004198,0.003250


In [8]:
# Define the cosine similarity between two vectors 
def cos_sim(a, b):
    """
    Takes 2 vectors a, b and returns the cosine similarity 
    """
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    return dot_product / (norm_a * norm_b)

In [9]:
def sim_tran_pattern(transaction_weights, pattern_weights):
    
    num_tran = transaction_weights.shape[1]
    num_p = pattern_weights.shape[1]
    
    sim_scores = pd.DataFrame(columns = pattern_weights.columns, index = transaction_weights.columns)
    for t in range(num_tran):
        for p  in range(num_p):
            #features_ind = (~transaction_weights.iloc[:,t].isna()) & (~author_weights.iloc[:,a].isna())
            tran =  transaction_weights.iloc[:, t]
            pattern = pattern_weights.iloc[:, p]
         
            #sim_scores.iloc[t,a] = cos_sim(tran, author)
            sim_scores.iloc[t,p]  = 1 - spatial.distance.cosine(tran, pattern)
    return sim_scores

### Show representative titles for a given author

In [10]:
sim_scores_author = sim_tran_pattern(transaction_weights, author_weights)

In [24]:
sim_scores_author.to_csv(path+"similarity_scores_of_transaction_to_author.csv", index = False)

In [25]:
author = authors[0]
r = 5
sorted_scores_author = sim_scores_author.sort_values(by = author, ascending = False)
sorted_scores_author.head(10)

,Ralf Steinmetz,Cheng-Wen Wu,Thomas S. Huang,Maja J. Mataric,K. Suzanne Barber,Josef Kittler,Gerald Sommer,Edwin R. Hancock,Masaru Kitsuregawa,Roberto Gorrieri,Nancy E. Reed,Bharat K. Bhargava,Sanjay Kumar Madria,Bill Hancock
1776,-0.945459,-0.952752,-0.950845,-0.946953,-0.953596,-0.94771,-0.951115,-0.952949,-0.94992,-0.952324,-0.952031,-0.953386,-0.952235,-0.953096
3030,-0.956988,-0.964138,-0.960658,-0.959555,-0.96527,-0.958469,-0.962903,-0.965561,-0.960531,-0.964402,-0.965173,-0.964456,-0.964293,-0.96537
1281,-0.958417,-0.956428,-0.955722,-0.954628,-0.961929,-0.957072,-0.955226,-0.961945,-0.961589,-0.95794,-0.960364,-0.954945,-0.958187,-0.957138
631,-0.95884,-0.956869,-0.959275,-0.957017,-0.959336,-0.957148,-0.958651,-0.959446,-0.95895,-0.959661,-0.955809,-0.958677,-0.959889,-0.955899
1200,-0.958951,-0.955824,-0.958253,-0.955301,-0.960088,-0.959257,-0.959121,-0.961769,-0.959574,-0.960468,-0.961347,-0.95608,-0.960537,-0.95783
3200,-0.959366,-0.959053,-0.959949,-0.959298,-0.961252,-0.959909,-0.960808,-0.961213,-0.962449,-0.961546,-0.959675,-0.960755,-0.96183,-0.956034
3952,-0.959366,-0.959053,-0.959949,-0.959298,-0.961252,-0.959909,-0.960808,-0.961213,-0.962449,-0.961546,-0.959675,-0.960755,-0.96183,-0.956034
2544,-0.959366,-0.959053,-0.959949,-0.959298,-0.961252,-0.959909,-0.960808,-0.961213,-0.962449,-0.961546,-0.959675,-0.960755,-0.96183,-0.956034
1010,-0.959517,-0.960926,-0.961784,-0.955901,-0.962862,-0.954646,-0.95775,-0.964097,-0.962526,-0.96167,-0.962785,-0.962583,-0.961946,-0.962482
3323,-0.960687,-0.963156,-0.964122,-0.960141,-0.964455,-0.962137,-0.9648,-0.966107,-0.96108,-0.966263,-0.964359,-0.964984,-0.966118,-0.966093


In [26]:
ind = sorted_scores_author.index[0:r]
rep_titles = pd.DataFrame(columns  = [author])
rep_titles[author] = D['title'].iloc[ind]
rep_titles

,Ralf Steinmetz
1776,Domain Name Based Visualization of Web Histori...
3030,Intelligent graphical user interface design ut...
1281,Realistic Force Feedback for Virtual Reality B...
631,Techniques for simulating difficult queueing p...
1200,Blackboard Segmentation Using Video Image of L...


In [27]:
rep_titles.to_csv(path + 'rep_titles_author_example1.csv', index = False)

### Calculate similarity between the transaction and the title

In [28]:
# Get title list 
titles = context_units.iloc[14:]["pattern"]
titles

14                   xml
15       extend abstract
16              workshop
17                  name
18                weight
             ...        
175              robocup
176             contextu
177            intension
178                 nest
179    distribut resourc
Name: pattern, Length: 166, dtype: object

In [29]:
title_weights = context_weights.iloc[:, 15:]
title_weights

,xml,extend abstract,workshop,name,weight,transmiss,use applic,guest introduct,autonomi,use transform,...,warp,movement,trade,reusabl,phylogeni,robocup,contextu,intension,nest,distribut resourc
0,0.003063,0.003177,0.003095,0.003314,0.003339,0.003415,0.002688,0.003037,0.003091,0.003858,...,0.003227,0.003914,0.003119,0.004322,0.003792,0.003134,0.003914,0.004453,0.003179,0.002649
1,0.003139,0.003244,0.003635,0.002905,0.002963,0.003993,0.003680,0.003095,0.003149,0.003430,...,0.003851,0.002292,0.003688,0.003796,0.003851,0.003743,0.003382,0.003910,0.004390,0.003230
2,0.003063,0.003177,0.003095,0.002847,0.003339,0.003415,0.003135,0.003037,0.003091,0.003363,...,0.003792,0.003331,0.003119,0.003737,0.003792,0.003684,0.003914,0.003851,0.003737,0.002649
3,0.002648,0.003177,0.003095,0.002847,0.002897,0.003415,0.002688,0.002570,0.002617,0.003363,...,0.002690,0.002779,0.003119,0.004322,0.003792,0.003134,0.003331,0.004453,0.003179,0.002649
4,0.003024,0.002801,0.003564,0.002937,0.002915,0.003487,0.003178,0.003149,0.003679,0.003430,...,0.003458,0.002531,0.003241,0.003402,0.004574,0.003880,0.003579,0.004071,0.003940,0.003402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.002528,0.002619,0.002973,0.002311,0.002775,0.002831,0.003013,0.002478,0.002984,0.002785,...,0.003134,0.002190,0.002550,0.003086,0.004262,0.004138,0.003238,0.003743,0.003629,0.003086
176,0.003220,0.003315,0.003233,0.003452,0.003497,0.003554,0.002808,0.003667,0.003212,0.003502,...,0.003914,0.004035,0.003759,0.003858,0.003914,0.003238,0.005328,0.003973,0.003283,0.002738
177,0.003139,0.003244,0.003635,0.003381,0.003415,0.003482,0.003202,0.003095,0.003149,0.003934,...,0.003278,0.003382,0.003178,0.004390,0.004453,0.003743,0.003973,0.005162,0.003796,0.003230
178,0.002993,0.002671,0.003487,0.002792,0.002834,0.003352,0.003532,0.003465,0.003525,0.003300,...,0.004322,0.002222,0.003064,0.003131,0.003737,0.003629,0.003283,0.003796,0.004862,0.003131


In [30]:
start = time.time()
sim_scores_title = sim_tran_pattern(transaction_weights, title_weights)
end = time.time()
print((end-start)/60)

3.84025768438975


In [31]:
sim_scores_title.to_csv(path+"similarity_scores_of_transaction_to_title.csv", index = False)

### Show representative titles for a given title

In [32]:
title = titles[10+15] # the same one used for definition
r = 5
sorted_scores_title = sim_scores_title.sort_values(by = title, ascending = False)
sorted_scores_title.head(10)

,xml,extend abstract,workshop,name,weight,transmiss,use applic,guest introduct,autonomi,use transform,...,warp,movement,trade,reusabl,phylogeni,robocup,contextu,intension,nest,distribut resourc
1776,-0.952545,-0.950688,-0.95266,-0.94647,-0.951724,-0.952336,-0.952957,-0.948504,-0.952347,-0.94973,...,-0.949892,-0.940608,-0.950546,-0.947096,-0.953766,-0.95482,-0.947849,-0.948456,-0.951598,-0.949649
1281,-0.959504,-0.953065,-0.960143,-0.96001,-0.955638,-0.958958,-0.957007,-0.958073,-0.960413,-0.961787,...,-0.955297,-0.95547,-0.957088,-0.955155,-0.95874,-0.955587,-0.95459,-0.958134,-0.956879,-0.958599
1200,-0.960302,-0.957883,-0.959149,-0.958179,-0.959027,-0.960496,-0.95593,-0.957784,-0.960094,-0.960261,...,-0.956135,-0.95751,-0.959916,-0.957444,-0.959516,-0.956587,-0.958427,-0.957223,-0.95464,-0.955594
631,-0.95687,-0.956716,-0.957652,-0.953876,-0.95751,-0.957796,-0.955905,-0.95332,-0.957593,-0.957812,...,-0.951879,-0.951644,-0.953952,-0.957306,-0.960775,-0.959498,-0.952751,-0.958671,-0.955614,-0.953317
3952,-0.958412,-0.958696,-0.960724,-0.955862,-0.959236,-0.959648,-0.95758,-0.952302,-0.953197,-0.957868,...,-0.951045,-0.958636,-0.959482,-0.957824,-0.962925,-0.960122,-0.960525,-0.960729,-0.952642,-0.955982
2544,-0.958412,-0.958696,-0.960724,-0.955862,-0.959236,-0.959648,-0.95758,-0.952302,-0.953197,-0.957868,...,-0.951045,-0.958636,-0.959482,-0.957824,-0.962925,-0.960122,-0.960525,-0.960729,-0.952642,-0.955982
3200,-0.958412,-0.958696,-0.960724,-0.955862,-0.959236,-0.959648,-0.95758,-0.952302,-0.953197,-0.957868,...,-0.951045,-0.958636,-0.959482,-0.957824,-0.962925,-0.960122,-0.960525,-0.960729,-0.952642,-0.955982
1930,-0.962359,-0.962367,-0.961636,-0.957602,-0.962989,-0.963135,-0.956993,-0.959076,-0.961479,-0.961272,...,-0.95799,-0.959796,-0.962921,-0.960925,-0.962957,-0.958496,-0.961869,-0.960715,-0.95643,-0.953834
2444,-0.958485,-0.957986,-0.958733,-0.958371,-0.958926,-0.95903,-0.958568,-0.954862,-0.95254,-0.960356,...,-0.95489,-0.958407,-0.956927,-0.959675,-0.956655,-0.956807,-0.957352,-0.960844,-0.956475,-0.954059
1010,-0.961604,-0.955509,-0.962424,-0.957741,-0.959403,-0.959796,-0.962635,-0.957405,-0.961308,-0.962609,...,-0.962926,-0.95496,-0.959618,-0.956285,-0.961408,-0.961933,-0.957287,-0.960874,-0.962866,-0.963522


In [33]:
ind = sorted_scores_title.index[0:r]
rep_titles2 = pd.DataFrame(columns  = [title])
rep_titles2[title] = D['title'].iloc[ind]
rep_titles2

,virtual realiti
1776,Domain Name Based Visualization of Web Histori...
1281,Realistic Force Feedback for Virtual Reality B...
1200,Blackboard Segmentation Using Video Image of L...
631,Techniques for simulating difficult queueing p...
3952,An approximate model for the computation of bl...


In [35]:
rep_titles2.to_csv(path + 'rep_titles_title_example1.csv', index = False)